In [17]:
import numpy as np
import pandas as pd
from importlib import reload
import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import shared
import config
import provider_yfinance as provider
import model
import plot

reload(shared)
reload(config)
reload(provider)
reload(model)
reload(plot)

plot.Notebook()

mpl.rcParams['figure.figsize'] = (16, 9)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['axes.unicode_minus']=False
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

# most recent data
cfg = config.get_config('^GDAXI')

# overwrite download_end_dt: use cached data
# config.save_config(cfg)

config> created config from file: './config.json'
config> config
        - base:
            - config_file_path: /mnt/c/notebooks/sandbox/config.json
        - datasets:
            - stocks: 30
            - benchmarks: 69
        - prepare:
            - data_start_dt: 2018-02-25
            - data_end_dt: 2020-01-26
            - cache_dir: /mnt/c/notebooks/sandbox/cache/20200126/
        - train:            
            - window_trading_days: [3, 5, 21, 35, 50]
            - lag_trading_days: [1, 2, 3, 4, 5]
            - label_max_high_weight: 3.0
            - label_max_close_weight: 1.0
            - settings: 5
        - model:
            - max_samples: 40
            - batch_size: 32
            - learning_rate: 0.1
            - learning_rate_decay: 0.75
            - lstm_hidden_size: 256
            - early_stopping_patience: 10
            - validation_monitor: val_mean_squared_error
            - max_epochs: 1000
            - base_dir: /mnt/c/notebooks/sandbox/model/202

In [18]:
%%time
eval_result = model.validate_model(cfg, validate_dt='2020-01-07')

 lookback_3-label_1
eval> lookback_3-label_1 - 1COV.DE - 2020-01-07 ...
WARN model> failed to load optimizer weights from '/mnt/c/notebooks/sandbox/model/20200107/lookback_3-label_1/1COV.DE/optimizer_weights.pkl': $You called `set_weights(weights)` on optimizer Nadam with a  weight list of length 27, but the optimizer was expecting 28 weights. Provided weights: [185, array([ 3.5113153e-06,  1.5836325e-04,  1.42...
WARN model> failed to load optimizer weights from '/mnt/c/notebooks/sandbox/model/20200107/lookback_3-label_1/1COV.DE/optimizer_weights.pkl': $You called `set_weights(weights)` on optimizer Nadam with a  weight list of length 27, but the optimizer was expecting 28 weights. Provided weights: [185, array([ 3.5113153e-06,  1.5836325e-04,  1.42...
WARN model> failed to load optimizer weights from '/mnt/c/notebooks/sandbox/model/20200107/lookback_3-label_1/optimizer_weights.pkl': $You called `set_weights(weights)` on optimizer Nadam with a  weight list of length 27, but the optimi

KeyboardInterrupt: 

In [19]:
%%time
(df_rank, df_eval) = model.rank_model_by_weighted_score(cfg)
df_rank

FileNotFoundError: [Errno 2] File b'/mnt/c/notebooks/sandbox/model/20200126/model_eval.tsv' does not exist: b'/mnt/c/notebooks/sandbox/model/20200126/model_eval.tsv'

In [40]:
df = df_eval[df_eval.submodel.isin([
    'lookback_3-label_1',
    'lookback_3-label_2',
    'lookback_5-label_1',
    'lookback_21-label_5',
    'lookback_100-label_7'    
])].groupby('ticker_name').agg('mean').sort_values('y_predicted', ascending=False)[:10]
df = df[df.y_predicted > 1.]
top3_stocks = df.index.tolist()
df

,y_predicted,y_actual,mdl_loss,mdl_mae,mdl_mse,mdl0_loss,mdl0_mae,mdl0_mse,scores,scores0,scores_sum
ticker_name,,,,,,,,,,,
WDI.DE,2.752,5.240,37.508652,3.542301,37.508651,41.375600,4.182084,41.375600,3.0,1.4,4.4
1COV.DE,2.372,2.930,3.391878,1.564816,3.391878,2.069866,1.103788,2.069866,4.6,5.0,9.6
FME.DE,2.100,2.154,0.887311,0.712088,0.887311,1.741340,1.055166,1.741340,5.4,3.0,8.4
HEN3.DE,1.830,1.822,1.409984,0.949741,1.409984,1.117052,0.753138,1.117052,4.2,4.4,8.6
HEI.DE,1.754,1.536,2.382038,1.442502,2.382038,1.434784,1.161316,1.434784,3.0,1.8,4.8
BMW.DE,1.646,0.886,2.023732,0.989210,2.023732,2.442053,1.305365,2.442053,4.6,2.2,6.8
IFX.DE,1.594,3.366,5.309187,2.121759,5.309186,3.153440,1.451604,3.153440,5.4,5.2,10.6
SAP.DE,1.552,2.292,1.304195,0.981571,1.304195,1.289976,0.896865,1.289976,5.4,4.8,10.2
CON.DE,1.444,3.702,6.079030,2.262204,6.079030,4.836563,1.888777,4.836563,4.6,5.0,9.6


In [42]:
df = df_eval[df_eval.submodel.isin([
    'lookback_3-label_1',
    'lookback_3-label_2',
    'lookback_5-label_1',
]) & df_eval.ticker_name.isin(top3_stocks)].groupby('ticker_name').agg('max').sort_values('y_predicted', ascending=False)
df = df[df.y_predicted > 1.]
df

,submodel,date,y_predicted,y_actual,mdl_loss,mdl_mae,mdl_mse,mdl0_loss,mdl0_mae,mdl0_mse,scores,scores0,scores_sum
ticker_name,,,,,,,,,,,,,
WDI.DE,lookback_5-label_1,2020-01-07,2.21,1.92,3.615715,1.901503,3.615715,5.493834,2.343893,5.493834,10,5,12
HEN3.DE,lookback_5-label_1,2020-01-07,1.23,2.05,3.375018,1.837122,3.375018,3.827623,1.956431,3.827623,10,8,18
FME.DE,lookback_5-label_1,2020-01-07,1.06,2.39,3.648036,1.909983,3.648036,4.933257,2.221094,4.933257,5,8,13
1COV.DE,lookback_5-label_1,2020-01-07,1.01,3.47,8.107730,2.847408,8.107730,7.129764,2.670162,7.129764,8,8,16
